In [ ]:
def getSectionfromRow(row):
    splitString = []
    sectionH = ""
    print("Inside function getSection", row[0])
    #for i in range(len(row)):
    if(len(row[0]) > 0):
        splitString = row[0].split(" ")
        if row[1] !="" and row[0] != "":
            sectionH = row[0]
            print("Section bcos there is a Column Header")
            return sectionH
        
        for j in range(len(splitString)):
            if splitString[j].upper().split()=="PART" or splitString[j].upper().split()=="SECTION":
                sectionH = row[0].split(":")[1]
                print("Section bcos of Part/Section")
                return sectionH
        print("String being checked", splitString[0])
        if splitString[0].find(".") != -1:
            print("After checking", splitString[0].split(".")[0])
            res = splitString[0].split(".")[0]
            res.replace(".", "")
            print("section being checked: ", res)
            #if res.isupper() == True or res.isdigit() == True:
            if res.isupper() ==True and res.isalpha() == True:
                sectionH = row[0].split(".")[1]
                print("Section bcos of Capital letter")
                return sectionH
    return ""      
            

def checkIfAllColumnHeadersAreBlank(row):
    nullCount = 0
    for i in range(len(row)):
        if row[i] == '':
            nullCount+=1
    if nullCount == len(row):
        return True
    else:
        return False

def formatStringForLIDesc(string):
    start = 0
    if string[0].isdigit() == True:
        start = string.find(" ") + 1
    res = string[start:]
    res = res.replace(".", "")
    res = res.replace("(", "")
    res = res.replace("0", "")
    print("formated result: ", res)
    return res
   


In [ ]:
import xlrd
import csv
import pandas as pd

#filename = "akhil_testxls3.xlsx"
filename = 'MA(BS)1c_Form(201801).csv'
path = 'D:/Akhil/Work/DOTS/RegAutomation/Akhil/HK/'

print("Hello Akhil 1")

In [ ]:
#Open the output file with list of Line Item Descriptions
print("Hello Opening Files")
with open(path+'output/LI_Experiment_'+filename, 'a', newline='') as output:
   newWriter = csv.writer(output)

   #Open the Input file
   with open(path+'input/Input-'+filename, newline='') as File:
       reader = csv.reader(File)

       print("Declaring Variables")
       #Variable Declarations
       sectionHeader=''
       prevSectionHeader=''
       sectionId=0
       sectionRow="N"
       noOfColumns=0
       maxColumns=0
       columnHeaderString=""
       columnHeaderSplitString=[]
       columnHeaderStringList=[]
       columnRow=[]
       previousColumnHeaderRow=[]
       lineItemId = 0
       lineItemDesc = ""
       lineItemMetadata = []
       lineItemMetadataList = []

       rowCounter = 1
       #Traverse through each row from the input CSV
       for row in reader:
           print('Row number ', rowCounter)
           rowCounter+=1
           tempSectionHeader =""

           tempSectionHeader = getSectionfromRow(row)
           rowSection = "N"
           print("tempSectionHeader ", tempSectionHeader)
           if(tempSectionHeader != ''):
               sectionHeader=tempSectionHeader
               if sectionHeader != prevSectionHeader:
                   sectionId+=1
                   prevSectionHeader = sectionHeader
                   rowSection = "Y"
                   print("The section ", sectionId, " is ", sectionHeader)
               
           colIncrement=1
           print("After Section Header, identifying Column Headers")
           
           #need to revisit this IF condition for identifying Headers
           #if(row[0] == '' and (len(row)>1 and len(row[1])>1 and row[1][0].isnumeric()) == False):
           #if row has section details or (len(row)>1 and len(row[1])>1 and row[1][0].isnumeric())==False:
           print(row[0].split(),'~',rowSection,'~',len(row),'~', len(row[1]), '~')
           if ((row[0].split() == [] or rowSection == "Y") and len(row)>1 and len(row[1])>0 ) :
           #assumption : the to be appended column should have at least one value in the first 5 cols
           #if len(row)>1:
               noOfColumns = len(row)-1
               if noOfColumns > maxColumns:
                   maxColumns = noOfColumns
                   print("Max Columns is ", maxColumns)
               columnHeaderString=""
               print("Inside simple IF condition")

               while (colIncrement <= noOfColumns):
                   columnHeaderString = columnHeaderString + row[colIncrement] + '~~'
                   colIncrement +=1
               
               columnHeaderStringList.append(columnHeaderString)
           else:
               print("Inside ELSE Condition")
               #This section merges Header columns in different rows into a Single Cell
               if len(columnHeaderStringList) > 0:
                   columnRow=[]
                   columnRow.append('') 
                   print("ColumnHeaderStringList has ", len(columnHeaderStringList))

                   for j in range(maxColumns):
                       columnHeaderString=""
                       print("Inside J loop with ", j)
                       for i in range(len(columnHeaderStringList)):
                           print("Inside I loop with ", i)
                           columnHeaderSplitString =columnHeaderStringList[i].split('~~')
                           if j < len(columnHeaderSplitString):
                               columnHeaderString = columnHeaderString + columnHeaderSplitString[j]

                       columnRow.append(columnHeaderString)
                       
                   if checkIfAllColumnHeadersAreBlank(columnRow):
                        columnRow = previousColumnHeaderRow
                        print("ColHeaders are Blank")
            
                   print("about to write to File")
                   if columnRow != previousColumnHeaderRow:
                       newWriter.writerow(columnRow)
                       previousColumnHeaderRow = columnRow
                       columnHeaderString = ""
                       columnHeaderSplitString = []
                       columnHeaderStringList = []
                       print("Column headers written to File")
                   print("done for now")

               if rowSection == "N" and row[0] != "":
                   print("This Row is not a Section header+",len(columnRow))
                   for k in range(len(columnRow)):
                       if row[k] == "" and columnRow[k] != "":
                           lineItemId += 1
                           lineItemDesc = formatStringForLIDesc(row[0]) + " " + formatStringForLIDesc(columnRow[k])
                           lineItemMetadata.append(sectionId)
                           lineItemMetadata.append(sectionHeader)
                           lineItemMetadata.append(lineItemId)
                           lineItemMetadata.append(lineItemDesc)

                           lineItemMetadataList.append(lineItemMetadata)
                           lineItemMetadata = []
                           row[k] = lineItemId
                           print("LineItemId is ", lineItemId)

               newWriter.writerow(row)
               print("Writing the Entire Row")                   

print("Gonna write the Metadat now")  
df = pd.DataFrame(lineItemMetadataList, columns = ['SectionID', 'SectionName', 'LineItemId', 'LineItemDesc'] )
df.to_csv(path+'output/LI_Metadata_'+filename, index = None, header = True)
print("Created the Metadata File")













In [ ]:
word = "Hello World"
print (word.find('l'))
print (word[2:])
word = "IID4"
print (word.isupper())
print (word.isdigit())
print (word.isalpha())

In [ ]:
word = "I."
print(word.find(".")) 
if word.find(".") != -1:
    print("its working : ", word.split(".")[0])

        